In [1]:
import numpy as np
import uproot
import h5py
import os
import torch
import torch_geometric.nn as PyG
from torch_geometric.transforms import Distance
from torch_geometric.data import InMemoryDataset as PyGDataset, Data as PyGData
from torch_geometric.data import Data
import math
import numba
import numpy as np
import torch
import h5py
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as PyG
from torch_geometric.transforms import Distance
from torch_geometric.data import DataLoader
from torch_geometric.data import Data as PyGData
from torch_geometric.data import Data
import sys, os
import subprocess
import csv, yaml
import math
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import torch.optim as optim
import argparse
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.tri as tri
# import numpy, numba, awkward, awkward.numba

In [2]:

# path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/delphes'
# graph_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/graph/'
# output_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/h5/'


path = '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/'

output_path = '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5_v2/'




In [3]:
resampling_model = '/users/yewzzang/work/HEPGNN_4top/result/210917_4top_1layer_re/'

In [4]:
sys.path.append("./python")

In [5]:
from model.allModel import *

model = torch.load(resampling_model+'/model.pth', map_location='cpu')
model.load_state_dict(torch.load(resampling_model+'/weight.pth', map_location='cpu'))
model = model.cuda()
device = 'cuda'

In [6]:
model

GNN1layer_re(
  (conv1): PointConvNet(
    (conv): PointConv(local_nn=Sequential(
      (0): Sequential(
        (0): Linear(in_features=7, out_features=64, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=128, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    ), global_nn=None)
  )
  (pool): PoolingNet(
    (net): Sequential(
      (0): Sequential(
        (0): Linear(in_features=131, out_features=128, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5

In [11]:
len(res)

430

In [7]:
res = []
for root, dirs, files in os.walk(path):
    rootpath = os.path.join(os.path.abspath(path), root)
    for file in files:
        filepath = os.path.join(rootpath, file)
        
        filetype = filepath.split('.')[-1]
        if filetype != 'h5': continue
  
        res.append(filepath)

In [8]:
res

['/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-6.h5',
 '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-89.h5',
 '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-93.h5',
 '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-68.h5',
 '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-10.h5',
 '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-60.h5',
 '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-77.h5',
 '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-63.h5',
 '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-27.h5',
 '/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg

In [9]:
import time
start = time.time()
for i in range(len(res)):

    file_name = res[i]
    cla = file_name.split('/')[-4].split('_')
    num = file_name.split('/')[-1].split('.')[-2]

    print(file_name)
    
    save_f_name = num

    save_path = output_path
    if not os.path.exists(save_path): os.makedirs(save_path)
    save_file_name = save_path + save_f_name + '.h5'

#     print(save_file_name)
### load data
    try:
        f = h5py.File(file_name,'r')
#         r_data = uproot.open(file_name)["Delphes"] 
#     r_data = uproot.open(file_name)["Delphes"]
    except KeyError:
        print("delphes 없음")
        pass
    
    
    
    
    
#     f = h5py.File(r_data,'r')

    nEvent = len(f['group']['pos'].get('pos'))

    f_pos = f['group']['pos'].get('pos')
    f_fea = f['group']['fea'].get('fea')
    f_edge = f['group']['edge'].get('edge')
    f_pos_list = []
    f_fea_list = []
    f_edge_list = []
    
    datalist = []
    weightlist = []
    weightslist = 0
    model.eval()
    for j in range(nEvent):
#                 f_pos_reshape = torch.reshape(f_pos[j],(-1,3))
        f_pos_reshape = f_pos[j].reshape(-1,3)
        f_fea_reshape = f_fea[j].reshape(-1,7)
        f_edge_reshape = f_edge[j].reshape(2,-1)



        weights = f_fea_reshape[:,6][0]/np.abs(f_fea_reshape[:,6][0])

        ### weights = w_ik
        ### weight = sigma_k / M_k
#         weightlist.append(weights)  
#         weightslist = weightslist + weights


    
        feats = torch.Tensor(f_fea_reshape[:,:4])
        poses = torch.Tensor(f_pos_reshape)
        edges = torch.Tensor(f_edge_reshape)
        edges = edges.type(dtype = torch.long)  
        
         
        data = PyGData(x = feats, edge_index = edges, pos = poses, batch = torch.from_numpy(np.zeros([len(feats)],dtype = np.int)))
 
        data = data.to(device)
        pred = model(data)
#         f['group']['fea']['fea'][j] = np.where(f['group']['fea']['fea'][j]==np.array(f['group']['fea']['fea'])[j].reshape(-1,7)[:,6][0],np.array(pred.detach().to('cpu')),f['group']['fea']['fea'][j])
        np.where(f['group']['fea']['fea'][j]==np.array(f['group']['fea']['fea'])[j].reshape(-1,7)[:,6][0],np.array(pred.detach().to('cpu')),f['group']['fea']['fea'][j])
    
        
    ff = h5py.File(save_file_name, mode='w')
    dt = h5py.special_dtype(vlen=np.dtype('float32'))
    gg = ff.create_group("group")
    
    pos = gg.create_group("pos")
    x_fea = gg.create_group("fea")
    edge_index = gg.create_group("edge")
    label = gg.create_group("label")
    

    pos.create_dataset('pos', (len(f_pos),), dtype=dt)
    pos['pos'][...] = np.array(f_pos)
    
    
    
    x_fea.create_dataset('fea', (len(f_pos),), dtype=dt)
    x_fea['fea'][...] = np.array(f['group']['fea']['fea'])
    
    
#     print(np.array(f_edge)[0].shape)
#     print(np.array(f_edge).reshape(-1,0).shape)
#     edge_index.create_dataset('edge', (len(f_pos),), dtype=dt)
#     edge_index['edge'][...] = np.array(f_edge)
    
#     label.create_dataset('label', (len(f_pos),), dtype=dt)
#     label['label'][...] = np.array(f['group']['label'].get('label'))
 
    ff.close()
    f.close()
    print(i,'fin')
print(time.time()-start)

/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-6.h5
0 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-89.h5
1 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-93.h5
2 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-68.h5
3 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-10.h5
4 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-60.h5
5 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-77.h5
6 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-63.h5
7 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-27.h5
8 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/outp

79 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-29.h5
80 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-79.h5
81 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-19.h5
82 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-70.h5
83 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-46.h5
84 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-67.h5
85 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-88.h5
86 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-82.h5
87 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2/output_ex1_v2-powheg-52.h5
88 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4t

154 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-73.h5
155 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-84.h5
156 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-197.h5
157 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-34.h5
158 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-140.h5
159 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-155.h5
160 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-97.h5
161 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-11.h5
162 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-po

227 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-2.h5
228 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-8.h5
229 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-115.h5
230 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-25.h5
231 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-167.h5
232 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-112.h5
233 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-182.h5
234 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-powheg-123.h5
235 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520/output_ex1_v2-po

300 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-118.h5
301 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-114.h5
302 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-68.h5
303 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-131.h5
304 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-10.h5
305 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-60.h5
306 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-77.h5
307 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-63.h5
308 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_

372 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-32.h5
373 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-50.h5
374 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-55.h5
375 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-128.h5
376 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-42.h5
377 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-62.h5
378 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-103.h5
379 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0/output_ex1_v2-powheg-44.h5
380 fin
/store/hep/users/yewzzang/4top_QCD_ttbar/data_graph/pt/4top_h5/ex1_v2_210520_0

In [10]:
ff.close()